In [ ]:
# 4.1 FewShotPromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

t = PromptTemplate.from_template("What is the capital of {country}")
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"]
# )

t.format(country="France")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# chat.predict("What do you know about France?")

# 예제의 형식을 지정하는 방법
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,                      # 프롬프트 템플릿
    examples=examples,                                  # 프롬프트에 입력될 값의 예시
    suffix="Human: What do you know about {country}?",  # 형식화된 모든 예제 마지막에 붙일 내용. 보통 사용자의 질문
    input_variables=["country"],
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI: 
        Here is what I know:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

In [8]:
# 4.2 FewShotChatMessagePromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

t = PromptTemplate.from_template("What is the capital of {country}")
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"]
# )

t.format(country="France")

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# chat.predict("What do you know about France?")

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])
# example_prompt = ChatPromptTemplate.from_template("Human: {country}\nAI: {answer}")

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,                      # 프롬프트 템플릿
    examples=examples,                                  # 프롬프트에 입력될 값의 예시
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers"),
    prompt,
    ("human", "What do you know about {country}?")
])

# prompt.format(country="Germany")

chain = final_prompt | chat

chain.invoke({"country": "Turkey"})


        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira
        

AIMessage(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira\n        ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--113227df-1326-43d3-a5ca-28175596a853-0')

In [ ]:
# 4.3 LengthBasedExampleSelector

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

t = PromptTemplate.from_template("What is the capital of {country}")
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"]
# )

t.format(country="France")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# chat.predict("What do you know about France?")

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


# 예제의 형식을 지정하는 방법
example_template = """
    Human: {question}
    AI: {answer}
"""

# example_prompt = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=180,
# )
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,                      # 프롬프트 템플릿
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",  # 형식화된 모든 예제 마지막에 붙일 내용. 보통 사용자의 질문
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [7]:
# 4.4 Serialization and Composition

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate


prompt = load_prompt('./prompt.json')
prompt = load_prompt('./prompt.yaml')

# prompt.format(country="Germany")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:
    
    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!
    
    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
    
    {example}
    
    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final, 
    pipeline_prompts=prompts
)

# full_prompt.format(
#     character='Pirate',
#     example_question='What is your location?',
#     example_answer='Arrrrg! That is a secret!! Arg Arg!!',
#     question='What is your favorite food?',
# )

chain = full_prompt | chat
chain.invoke({
    "character":'Pirate',
    "example_question":'What is your location?',
    "example_answer":'Arrrrg! That is a secret!! Arg Arg!!',
    "question":'What is your favorite food?',
})

Arrrrg! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, nothing beats the taste of the sea! Arrr!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, nothing beats the taste of the sea! Arrr!")

In [2]:
# 4.5 Caching

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
# set_debug(True)
set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make Italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppi

In [9]:
chat.predict("How do you make Italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a rolling pin or pasta machine.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta

In [5]:
# 4.6 Serialization

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)
    print(usage.total_cost)
    print(usage.total_tokens)
    print(usage.prompt_tokens)
    print(usage.completion_tokens)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and sugar with the cooked rice.
5. Transfer the mixture to a clean glass jar and cover it with a clean cloth.
6. Let the mixture ferment in a warm place for 3-4 days, stirring it once a day.
7. After 3-4 days, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store it in the refrigerator.
9. Serve the homemade soju chilled and enjoy responsibly. Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil

Instructions:
1. In a large mixing bowl, combine the warm water, sugar, and yeast. Let it sit for a

In [ ]:
# 4.6 Serialization

from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

In [8]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/Users/giljong-in/Workspace/FULLSTACK-GPT/.venv/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/giljong-in/Workspace/FULLSTACK-GPT/.venv/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})